# Split the Dataset

In [1]:
import pandas as pd
import sys
sys.path.append("../scripts")
from utils import Clean 
from load_data import LoadData

In [2]:
#initialize utilities
loader = LoadData()
cleaner = Clean()

In [3]:
#laod the original dataset
loader = LoadData()
df = loader.read_csv('../data/AdSmartABdata.csv')
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


We already have a raw dataset as version 1 of the data
We are going to create version 2 of the data by cleaning the raw dataset. First, we delete the raw dataset from the data folder

In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   8077 non-null   object
 1   experiment   8077 non-null   object
 2   date         8077 non-null   object
 3   hour         8077 non-null   int64 
 4   device_make  8077 non-null   object
 5   platform_os  8077 non-null   int64 
 6   browser      8077 non-null   object
 7   yes          8077 non-null   int64 
 8   no           8077 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 568.0+ KB


In [5]:
df = cleaner.run_pipeline(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   experiment   1243 non-null   object
 1   hour         1243 non-null   int64 
 2   platform_os  1243 non-null   int64 
 3   browser      1243 non-null   object
 4   day_of_week  1243 non-null   object
 5   brand        1243 non-null   object
 6   response     1243 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 68.1+ KB


../scripts\utils.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = pd.to_datetime(df['date']).dt.day_name().values
../scripts\utils.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['brand'] = makers
../scripts\utils.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

### Groupby browser


In [6]:
df.groupby('browser').count()

,experiment,hour,platform_os,day_of_week,brand,response
browser,,,,,,
Chrome,2,2,2,2,2,2
Chrome Mobile,695,695,695,695,695,695
Chrome Mobile WebView,227,227,227,227,227,227
Chrome Mobile iOS,1,1,1,1,1,1
Facebook,156,156,156,156,156,156
Mobile Safari,14,14,14,14,14,14
Mobile Safari UI/WKWebView,3,3,3,3,3,3
Samsung Internet,145,145,145,145,145,145


In [7]:
#Group by browser and store them on in a dictionary
by_browser = df.groupby('browser')
browsers = by_browser['experiment'].count() > 100
browsers = browsers[browsers==True].index
browser_dfs = {}
for browser in browsers:
    b_df = df.query(f"browser=='{browser}'").reset_index(drop=True)
    browser_dfs[browser] = b_df

In [8]:
browser_dfs.keys()

dict_keys(['Chrome Mobile', 'Chrome Mobile WebView', 'Facebook', 'Samsung Internet'])

In [9]:
#create version 2 of the data for chrome mobile
b_df=browser_dfs.get('Chrome Mobile')
b_df.head()


,experiment,hour,platform_os,browser,day_of_week,brand,response
0,exposed,16,6,Chrome Mobile,Saturday,generic,1
1,exposed,8,6,Chrome Mobile,Monday,generic,0
2,control,15,6,Chrome Mobile,Friday,generic,0
3,exposed,2,6,Chrome Mobile,Friday,generic,0
4,control,15,6,Chrome Mobile,Friday,generic,1


In [10]:
b_df.to_csv('../data/AdSmartABdata.csv')

In [11]:
#create version 3 of the data for Chrome Mobile WebView
b_df=browser_dfs.get('Chrome Mobile WebView')
b_df.head()

,experiment,hour,platform_os,browser,day_of_week,brand,response
0,exposed,2,6,Chrome Mobile WebView,Sunday,generic,0
1,exposed,6,6,Chrome Mobile WebView,Sunday,generic,0
2,exposed,20,6,Chrome Mobile WebView,Thursday,known brand,0
3,control,2,6,Chrome Mobile WebView,Monday,known brand,0
4,exposed,11,6,Chrome Mobile WebView,Saturday,known brand,1


In [12]:
b_df.to_csv('../data/AdSmartABdata.csv')